In [27]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import torch

In [28]:
torch.manual_seed(42)

In [29]:
torch.manual_seed(42)
n = 100
t = np.linspace(0, 1, n)
elevation_true = 0.8 * ((np.sin(50*t) + np.sin(10*t))/2)+ 0.1 * t   # Profilo di elevazione reale
elevation_measured = elevation_true + 0.1 * torch.rand(n).numpy()  # Dati misurati con rumore

In [30]:
# Operatori differenziali discreti
D = np.eye(n) - np.eye(n, k=-1)
D = D[:-1, :]  # Prima derivata discreta

D2 = np.eye(n) - 2 * np.eye(n, k=1) + np.eye(n, k=2)
D2 = D2[:-2, :]  # Seconda derivata discreta

In [31]:
ds = []

In [32]:
for i in range(1000):

	# Simulazione dei dati


	# Variabile di ottimizzazione
	x = cp.Variable(n)



	# Parametri del problema
	rho_param = np.random.rand()  # Peso della regolarizzazione sulla pendenza
	kappa_max = np.random.rand()   # Limite sulla curvatura

	# Funzione obiettivo
	objective = cp.Minimize(cp.sum_squares(x - elevation_measured))

	# Vincoli
	constraints = []

	# Vincoli sulla curvatura
	constraints += [cp.sum_squares(D2 @ x) <= kappa_max*np.linalg.norm(D2@elevation_measured)**2]
	constraints += [cp.sum_squares(D @ x) <= rho_param*np.linalg.norm(D@elevation_measured)**2]

	# Vincoli sui limiti di elevazione (se noti)
	elevation_min = np.min(elevation_measured) - 0.1
	elevation_max = np.max(elevation_measured) + 0.1
	#constraints += [x >= elevation_min]
	#constraints += [x <= elevation_max]

	# Risoluzione del problema
	prob = cp.Problem(objective, constraints)
	prob.solve()

	# Soluzione ottimale
	x_hat = x.value

	if prob.status == 'optimal': ds.append([np.array(rho_param), np.array(kappa_max), x.value, constraints[0].dual_value, constraints[1].dual_value])

/Users/carmine/miniforge3/envs/KKT-NN/lib/python3.12/site-packages/cvxpy/problems/problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [33]:
(x.value).shape

(100,)

In [34]:
constraints[0].dual_value

array([27.16851052])

In [35]:
np.concatenate([x.value, constraints[0].dual_value]).shape

(101,)

In [36]:
constraints[1]

Inequality(Expression(CONVEX, NONNEGATIVE, ()))

In [37]:
constraints[1].dual_value

array([1.22242205e-09])

In [38]:
constraints[0].dual_value

array([27.16851052])

In [39]:
np.linalg.norm(D2@x.value)**2 - kappa_max*np.linalg.norm(D2@elevation_measured)**2

2.1411775130708577e-11

In [40]:
np.linalg.norm(D@x.value)**2 - rho_param*np.linalg.norm(D@elevation_measured)**2

-1.5964355778896566

In [41]:
D.shape

(99, 100)

In [42]:
np.linalg.norm(2*(x.value - elevation_measured) + 2*constraints[1].dual_value*(D.T@(D@x.value))+ 2*constraints[0].dual_value*(D2.T@(D2@x.value)))

4.496204761653374e-05

In [43]:
constraints[1].dual_value*(np.linalg.norm(D@x.value)**2 - rho_param*np.linalg.norm(D@elevation_measured)**2)

array([-1.95151806e-09])

In [44]:
constraints[0].dual_value*(np.linalg.norm(D2@x.value)**2 - kappa_max*np.linalg.norm(D2@elevation_measured)**2)

array([5.81726038e-10])

In [45]:
ds

[[array(0.86486682),
  array(0.21686024),
  array([ 0.22040175,  0.33425285,  0.42770306,  0.47913391,  0.47180489,
          0.41036928,  0.31319112,  0.21091407,  0.13276163,  0.10516369,
          0.14614514,  0.24877883,  0.39360638,  0.5479115 ,  0.67769736,
          0.75209701,  0.75399899,  0.68018375,  0.54905312,  0.39254297,
          0.244856  ,  0.13976759,  0.0975057 ,  0.12231544,  0.19916881,
          0.30542557,  0.40706981,  0.47038537,  0.47189956,  0.40183691,
          0.27116877,  0.10037257, -0.07309293, -0.21746356, -0.3036792 ,
         -0.31726647, -0.2655752 , -0.16964465, -0.06253892,  0.02393915,
          0.06323065,  0.0385307 , -0.04873599, -0.18380023, -0.33852323,
         -0.47596361, -0.5615669 , -0.57829358, -0.52310544, -0.40977688,
         -0.26137957, -0.11029994,  0.00842577,  0.06970957,  0.05995378,
         -0.01074657, -0.11668617, -0.22266048, -0.29329443, -0.30368625,
         -0.2388622 , -0.10685036,  0.06973766,  0.25399264,  0.410401

In [46]:
from pickle import dump
dump(ds, open("smooth.pkl", "wb"))

In [47]:
x.value.shape

(100,)

In [48]:
D2.shape

(98, 100)

In [49]:
np.linalg.norm(D2@elevation_measured)**2

1.084829414601063

In [50]:
np.linalg.norm(D2 @ elevation_true)**2

0.49741764541027556

In [51]:
np.max(D2 @ elevation_true)

0.10242222754502994

In [52]:
np.linalg.norm(D2 @ x.value)

0.2517487945720737